In [1]:
import json
import csv
import requests
import urllib.parse
import pickle
from importlib import reload

#import translatorpy
from translatorpy.trapigraph import TrapiGraph
from translatorpy.translatorquery import TranslatorQuery
from translatorpy import utilities as translator_util

In [14]:
key_clusters = [2]#,1,94,105]

candidate_data = []
with open("../data/tox21_cluster_compound_target_candidates.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if (int(row['cluster']) in key_clusters) & (row['tier']=='Tier 3'):
            candidate_data.append(row)

all_candidate_genes = set([i['target_gene'] for i in candidate_data])
#translator_gene_names = translator_util.translate_node_name(all_candidate_genes,'NCBIGene',cutoff=True)
with open('../data/gene_names.pkl', 'rb') as handle:
    translator_gene_names = pickle.load(handle)

genename2ncbicurie = {i[0]:i[1] for i in translator_gene_names}

all_candidate_compounds = set([i['sample_name'] for i in candidate_data])
#translator_compound_names = translator_util.translate_node_name(all_candidate_compounds,'PUBCHEM.COMPOUND')

#with open('data/compound_names.pkl', 'wb') as handle:
#    pickle.dump(translator_compound_names,handle,protocol=pickle.HIGHEST_PROTOCOL)

with open('../data/compound_names.pkl', 'rb') as handle:
    translator_compound_names = pickle.load(handle)

samplename2pubchemcurie = {i[0]:i[1] for i in translator_compound_names}


In [15]:
len(candidate_data)

77

In [3]:

for target_candidate in [x for x in candidate_data if (x['target_gene']=='ERBB2') & (x['sample_name']=='Posaconazole')]:
    if target_candidate['sample_name'] in samplename2pubchemcurie:
        compound=samplename2pubchemcurie[target_candidate['sample_name']]
    else:
        cid = target_candidate['pubchem_cid']
        compound = f'PUBCHEM.COMPOUND:{cid}'    
    gene = genename2ncbicurie[target_candidate['target_gene']]
    direct_edge_list = [[compound,gene,'biolink:related_to']]
    indirect_edge_list = [[compound,'biolink:NamedThing','biolink:related_to'],['biolink:NamedThing',gene,'biolink:related_to']]
    node_categories = {compound:['biolink:ChemicalEntity'],gene:['biolink:Gene']}
    candidate_direct_trapi = TrapiGraph(direct_edge_list,format='SOP',node_data=node_categories)
    candidate_indirect_trapi = TrapiGraph(indirect_edge_list,format='SOP',node_data=node_categories)


In [4]:
candidate_indirect_trapi.query

{'message': {'query_graph': {'edges': {'e00': {'subject': 'n00',
     'object': 'n01',
     'predicates': ['biolink:related_to']},
    'e01': {'subject': 'n01',
     'object': 'n02',
     'predicates': ['biolink:related_to']}},
   'nodes': {'n00': {'ids': ['PUBCHEM.COMPOUND:468595'],
     'categories': ['biolink:ChemicalEntity']},
    'n01': {'categories': ['biolink:NamedThing']},
    'n02': {'ids': ['NCBIGene:2064'], 'categories': ['biolink:Gene']}}}}}

In [5]:
#myquery._TranslatorQuery__submit_to_ars(myquery.query_graph.query)
myquery = TranslatorQuery()
myquery.query(candidate_indirect_trapi,delay=30)


Still Running
Still Running
Still Running
Still Running
Still Running
Still Running
Still Running
Still Running
Still Running
Done
Done
Done ara-aragorn 399
Done
Done ara-arax 0
Done
Done ara-bte 177
Error
Error ara-unsecret 0
Unknown
Unknown kp-genetics 0
Error
Error kp-molecular 0
Done
Done ara-explanatory 3
Done
Done ara-improving 1
Done
Done kp-cam 0
Done
Done kp-textmining 40
Done
Done kp-openpredict 0
Done
ARS Error kp-icees 0
Done
Done kp-chp 0
Error
Error kp-cohd 0
Done
ARS Error kp-icees-dili 0


In [6]:
myquery.to_tsv('../data/translator_example_results.tsv')